# 欢迎使用 ✨🦋 illufly

[![PyPI version](https://img.shields.io/pypi/v/illufly.svg)](https://pypi.org/project/illufly/)

`illufly` 是 `illution butterfly` 的缩写，中文为"幻蝶"。

**illufly** 是一个具有自我进化能力的 Agent 框架，目标是：`基于自我进化，快速创作价值`。

illufly 被设计为在意图猜测、问答经验、资料召回率、工具规划能力等各种场景下都具有自我进化能力。<br>
本文作为开始，一步一步讲述各种场景下的自我进化如何实现。

**请注意:** 由于 illufly 还处于开发状态，为了加强自我进化能力，框架的一些概念会不断更新，使用时请锁定版本。

## 1 从内置的 RAG 能力开始讲起

illufly 使用时简单、直接、快速，但创造价值的场景却很丰富。<br>
从 illufly.chat 导入一个封装好的大模型是最常见的开始方式。

```python
from illufy.chat import ChatQwen
```

ChatQwen 是 ChatAgent 子类。<br>
这一行代码很简单，但你会越来越惊奇地发现，这个 Agent 已经具备很多魔法能力。

### 1.1 连续对话

**首先是连续对话能力：**

In [1]:
from illufly.chat import ChatQwen
qwen = ChatQwen()

qwen("请你帮我写封一句话情书，深情又逗比的那种")

在这浩瀚的宇宙中，我愿意做你的小行星，绕着你转，偶尔撞进你的心里，制造一点浪漫的火花和小小的混乱。


'在这浩瀚的宇宙中，我愿意做你的小行星，绕着你转，偶尔撞进你的心里，制造一点浪漫的火花和小小的混乱。'

实际上，上述代码已经内置了一些功能特性：

- **流式输出** 内置流输出
- **支持连续对话** 问答过程是有记忆的，可以连续对话

**查看对话记忆：**

In [2]:
qwen.memory

[{'role': 'user', 'content': '请你帮我写封一句话情书，深情又逗比的那种'},
 {'role': 'assistant',
  'content': '在这浩瀚的宇宙中，我愿意做你的小行星，绕着你转，偶尔撞进你的心里，制造一点浪漫的火花和小小的混乱。'}]

### 1.2 内置 RAG 支持

使用 RAG（检索增强生成）是开发大模型应用时的常见场景。<br>
illufly 内置了一些 RAG 实现策略，最简单的就是直接将背景知识添加到 Agent 中。

**构建最朴素的 RAG 应用：**

In [37]:
from illufly.types import Messages
Messages(PromptTemplate(text="请你帮我写封一句话情书，深情又逗比的那种, 已有知识:{{knowledge}}"), style="text")[-1]

Message(role=system, content=请你帮我写封一句话情书，深情又逗比的那种, 已有知识:)

In [2]:
from illufly.chat import ChatQwen
from illufly.types import PromptTemplate

# 声明大模型实例
qwen = ChatQwen(knowledge=[
    "我的女朋友名字叫林徽因，我喜欢叫她「银子」",
    "她喜欢叫我「金子」",
])

# 使用模板
qwen(
    PromptTemplate(
        text="请你帮我写封一句话情书，深情又逗比的那种, {{knowledge}}",
        binding_map={"knowledge": lambda: qwen.get_knowledge()}
    )
)
qwen.memory

"亲爱的银子，虽然你的价值连城，但我愿意用我所有的金子，换取你的一笑。"


[{'role': 'system',
  'content': "请你帮我写封一句话情书，深情又逗比的那种, ['我的女朋友名字叫林徽因，我喜欢叫她「银子」', '她喜欢叫我「金子」']"},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '"亲爱的银子，虽然你的价值连城，但我愿意用我所有的金子，换取你的一笑。"'}]

**将资料保存到文件并根据问题召回：**

illufly 也支持传统的 RAG 流程：将文档切分成多个片段，再通过向量模型比较问题和文档片段，这个过程被称为「召回」，也就是从数据库中查找到文本相似的那部份文档片段。

你可以把资料整理为 markdown 文件，放入指定位置，比如 `./docs/gf.md` 中，然后使用向量模型嵌入文档，再使用向量数据库检索，最后加载到大模型的提示语中。

在 illufly 框架中，这个过程依然非常简洁，你只负责声明实例就可以，其余的交给 illufly 实现。

In [5]:
from illufly.rag import TextEmbeddings, FaissDB
from illufly.chat import ChatQwen

# 声明向量数据库并加载指定位置的文档
db = FaissDB(embeddings=TextEmbeddings(), top_k=3)
db.load("./docs")

# 声明大模型实例
qwen = ChatQwen(knowledge=[db])

# 使用
qwen("请你帮我写封一句话情书，深情又逗比的那种")
qwen.memory

亲爱的银子，你是我心中的小白兔，不仅因为你的灵动可爱，更因为你总能让我这个大笨“金子”为你跳起最不协调却充满爱的舞蹈。爱你，是我最美的决定。


[{'role': 'user',
  'content': '回答时请参考已有知识：\n@knowledge\n我的女朋友名字叫林徽因，我喜欢叫她「银子」,\n她喜欢叫我「金子」,\n林徽因特别喜欢小兔子\n\n**Question**\n林徽因和她的喜好\n\n**Knowledge**\n林徽因是用户的女朋友，用户私下里称她为“因子”。她称呼用户为“金子”，并且喜欢小白兔。\n\n**Question**\n林徽因的姓名及爱好\n\n**Knowledge**\n林徽因是用户的女朋友，她喜欢小白兔。\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': '请你帮我写封一句话情书，深情又逗比的那种'},
 {'role': 'assistant',
  'content': '亲爱的银子，你是我心中的小白兔，不仅因为你的灵动可爱，更因为你总能让我这个大笨“金子”为你跳起最不协调却充满爱的舞蹈。爱你，是我最美的决定。'}]

### 1.3 在对话中自主进化

为了让大模型能够理解对话的背景，采用 RAG 策略的确是好办法，但管理 RAG 文档资料有些繁琐，涉及到文档准备、确认、加载、切分、检索等很多细节。你希望大模型记住的知识也许是未经整理的、碎片化的，这让 RAG 文档资料很难管理。

illufly 提供自我进化能力，其中之一就是在对话过程中学习知识。

在对话中获得经验需要使用 ChatLearn 子类。

In [6]:
from illufly.chat import ChatQwen
from illufly.learn import ChatLearn

talker = ChatLearn(ChatQwen())

In [7]:
talker("我跟你说说我的女朋友")

[AGENT] >>> Node 1: Scribe
当然可以，我很乐意听你分享关于你女朋友的事情。你可以告诉我一些她的兴趣爱好、你们是如何认识的，或者任何你觉得有趣的事情。


'当然可以，我很乐意听你分享关于你女朋友的事情。你可以告诉我一些她的兴趣爱好、你们是如何认识的，或者任何你觉得有趣的事情。'

In [8]:
talker("她叫林徽因，我私下里叫她`银子`，她就叫我`金子`")

[AGENT] >>> Node 1: Scribe
听起来你们之间的昵称很有意思。林徽因这个名字也很好听。你们是怎么认识的呢？有没有什么特别的故事？


'听起来你们之间的昵称很有意思。林徽因这个名字也很好听。你们是怎么认识的呢？有没有什么特别的故事？'

In [9]:
talker("你帮我总结吧")

[AGENT] >>> Node 1: Scribe
**思考**
- 对话中的关键信息包括：用户女朋友的名字叫林徽因，用户私下里叫她“银子”，她则叫用户“金子”。
- 对比已有的知识，没有发现与上述信息相关的`@knowledge`开头的内容。
- 这些信息是新的，且不与任何已知信息冲突。
- 信息包含新增的内容，如昵称和名字。

**决定**
- 没有发现与`@knowledge`开头的已有知识存在冲突。
- 没有发现与`@knowledge`开头的已有知识基本重复的内容。
- 因此，返回`question`和`knowledge`。

**结论**

<question>
用户女朋友的名字及昵称是什么？
</question>

<knowledge>
用户女朋友的名字叫林徽因，用户私下里称呼她为“银子”，而她则称呼用户为“金子”。
</knowledge>
[AGENT] >>> Node 3: Fetch_FAQ
[FAQ] 保存知识到[099671-7364-0000]：用户女朋友的名字及昵称是什么？ -> 用户女朋友的名字叫林徽因，用户私下里称呼她为“银子”，而她则称呼用户为“金子”。


'**思考**\n- 对话中的关键信息包括：用户女朋友的名字叫林徽因，用户私下里叫她“银子”，她则叫用户“金子”。\n- 对比已有的知识，没有发现与上述信息相关的`@knowledge`开头的内容。\n- 这些信息是新的，且不与任何已知信息冲突。\n- 信息包含新增的内容，如昵称和名字。\n\n**决定**\n- 没有发现与`@knowledge`开头的已有知识存在冲突。\n- 没有发现与`@knowledge`开头的已有知识基本重复的内容。\n- 因此，返回`question`和`knowledge`。\n\n**结论**\n\n<question>\n用户女朋友的名字及昵称是什么？\n</question>\n\n<knowledge>\n用户女朋友的名字叫林徽因，用户私下里称呼她为“银子”，而她则称呼用户为“金子”。\n</knowledge>'

### 1.4 使用在对话中获得的经验

In [10]:
from illufly.rag import FaissDB, TextEmbeddings
from illufly.chat import ChatQwen

db = FaissDB(embeddings=TextEmbeddings(), top_k=3)
qwen = ChatQwen(knowledge=[db])

qwen("你知道我女朋友叫什么吗？有什么爱好?")

你的女朋友叫林徽因，她喜欢小白兔。另外，你私下里称呼她为“银子”或“因子”，而她则称呼你为“金子”。


'你的女朋友叫林徽因，她喜欢小白兔。另外，你私下里称呼她为“银子”或“因子”，而她则称呼你为“金子”。'

### 1.5 管理经验数据

illufly 的设置很多都是通过环境变量来指定的。<br>
在 python 中你可以通过 dotenv 来管理环境变量的设置，也可以通过 docker 或 python 的 os 模块来指定。

**使用 config 模块的 get_env() 可以查看经验目录的默认值**

对于不同的操作系统来说，这个目录位置可能有所不同，但默认情况下这应该是一个临时目录。

In [12]:
from illufly.config import get_env

# 如果不带参数，就返回所有环境变量的默认值
get_env("ILLUFLY_CHAT_LEARN")

'/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/__ILLUFLY__/CHART_LEARN'

**如果你不喜欢这个目录可以改为其他位置。不过在此之前，你也可以将已有经验迁移过来：**

In [13]:
qwen.clone_chat_learn("./XP")

'从 /var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/__ILLUFLY__/CHART_LEARN 拷贝到 ./XP 完成，共克隆了 3 个文件。'

**你可以通过 os.environ 来指定环境变量的值，设定新的经验存储目录：**

In [14]:
import os
os.environ["ILLUFLY_CHAT_LEARN"] = "./XP"
get_env("ILLUFLY_CHAT_LEARN")

'./XP'

上面简单介绍了基于文档资料的 RAG 和基于经验的 RAG 实现。<br>
接下来，继续介绍 illufly 中对于流行的智能体论文的实践和内置支持。

## 2 单智能体和工具回调

illufly 的 ChatAgent 天然具有使用工具的能力，可以直接作为单智能体使用。

### 2.1 所有 ChatAgent 都是 OpenAI 工具回调风格的智能体

在`illufly`中，所有对话智能体内置支持工具回调，只需要提供`tools`参数。<br>
而普通 python 函数即可当作工具使用。

**以下示例是定义工具和使用工具的过程：**

In [17]:
from illufly.chat import ChatQwen

def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qwen = ChatQwen(tools=[get_current_weather])

qwen("今天广州可以晒被子吗", verbose=False)
qwen.memory



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_8fc2bc07fb6d478ab1eb97", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
今天广州是晴天，非常适合晒被子哦。


[{'role': 'user', 'content': '今天广州可以晒被子吗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8fc2bc07fb6d478ab1eb97',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_8fc2bc07fb6d478ab1eb97',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '今天广州是晴天，非常适合晒被子哦。'}]

### 2.2 其他单智能体实现

illufly 内置实现了 ReAct、ReWoo、Plan and Solve 等流行的单智能体论文的实践。<br>

| FlowAgent子类 | 推理方式 | 论文来源 |
|:----|:--------|:------------|
|ReAct|一边推理一边执行|[ReAct](https://arxiv.org/abs/2210.03629) |
|ReWOO|一次性规划所有步骤后一起执行|[ReWOO](https://arxiv.org/abs/2305.18323) |
|PlanAndSolve|一边修订总体计划一边执行|[Plan-and-Solve](https://arxiv.org/abs/2305.04091) |

**illufly 如何实现工具回调能力的自我进化呢？** <br>
这是一个重要但复杂的话题，本文作为入门教程不展开讲述。

In [1]:
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "重庆"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是明确的城市名称。"""
    return f'{city}今天暴雨'

def booking(request: str):
    """你出差时，我可以帮你安排好到达地点后的酒店、出行等一切事宜"""
    return '我已经帮你预订好酒店，祝你出差顺利'

search = ChatQwen(
    name="搜索小能手",
    description="我擅长互联网资料收集",
    enable_search=True,
    memory=('system', '你是一个互联网资料收集专家,根据提问的问题整理好收集的资讯')
)

In [2]:
search.tool_desc

{'type': 'function',
 'function': {'name': '搜索小能手',
  'description': '我擅长互联网资料收集',
  'parameters': {'type': 'object',
   'properties': {'prompt': {'type': 'string', 'description': '详细描述用户问题'}},
   'required': ['prompt']}}}

**首先，直接使用 OpenAI 工具回调风格的智能体：**

In [4]:
qwen = ChatQwen(tools=[get_city, get_weather, booking, search])
qwen("最近哪个节目里的演员北大毕业的多？")



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_ebbff9c4a62148ad9e036d", "type": "function", "function": {"name": "搜索小能手", "arguments": "{\"prompt\": \"最近哪个节目里的演员北大毕业的多？\"}"}}]
最近，脱口秀类节目《脱口秀大会》中出现了多位毕业于北京大学的演员。例如，李雪琴和鸟鸟都是该节目的参与者，两人均拥有北京大学的教育背景：

- **李雪琴**：毕业于北京大学新闻与传播学院，是一位知名的脱口秀演员和网络红人。
- **鸟鸟**：本名赵晓卉，北京大学中文系硕士研究生，本科毕业于吉林大学，加入脱口秀行业前曾从事创意写作工作。

这些具有高学历背景的演员在节目中展现出了独特的视角和才华，受到了观众的广泛好评。《脱口秀大会》也因此被认为是近期聚集了较多北大毕业生的综艺节目之一。

最近，脱口秀类节目《脱口秀大会》中出现了多位毕业于北京大学的演员。比如，李雪琴和鸟鸟：

- **李雪琴**：毕业于北京大学新闻与传播学院，是知名脱口秀演员及网络红人。
- **鸟鸟**（本名赵晓卉）：北京大学中文系硕士研究生，本科毕业于吉林大学，在加入脱口秀行业之前曾从事创意写作工作。

这些具有高学历背景的演员在节目中展现出了独特的视角和才华，受到了观众的广泛好评。因此，《脱口秀大会》可以认为是近期聚集了较多北大毕业生的综艺节目之一。


'最近，脱口秀类节目《脱口秀大会》中出现了多位毕业于北京大学的演员。比如，李雪琴和鸟鸟：\n\n- **李雪琴**：毕业于北京大学新闻与传播学院，是知名脱口秀演员及网络红人。\n- **鸟鸟**（本名赵晓卉）：北京大学中文系硕士研究生，本科毕业于吉林大学，在加入脱口秀行业之前曾从事创意写作工作。\n\n这些具有高学历背景的演员在节目中展现出了独特的视角和才华，受到了观众的广泛好评。因此，《脱口秀大会》可以认为是近期聚集了较多北大毕业生的综艺节目之一。'

**然后，使用 ReAct 单步推理智能体：**

In [5]:
flow = ReAct(ChatQwen(tools=[get_city, get_weather, booking, search]))
flow("最近哪个节目里的演员北大毕业的多?")

[AGENT] >>> Node 1: planner
**思考**  
目前我没有足够的信息来确定最近哪个节目中的演员是北京大学的毕业生。因此，我需要进行网络搜索以获取相关信息。

**行动**  
Step1: 搜索最近哪些节目里有北大毕业的演员. #E1 = 搜索小能手[{"prompt": "最近哪个节目里的演员北大毕业的多"}]
最近，《非正式会谈》这档节目中出现了多位北京大学毕业的嘉宾。例如，节目中提到的一位飞行嘉宾纪向宁，正在北京大学攻读艺术系研究生，并且以439分的成绩位列北京大学艺术系研究生第一名。此外，该节目因其广泛的议题讨论和嘉宾的多样性而受到关注，其中不乏高学历背景的参与者，包括来自北京大学等知名学府的人士。

另外，脱口秀类节目《脱口秀大会》也是一个值得关注的例子，该节目中有多位演员拥有北京大学的教育背景，如李雪琴和鸟鸟（赵晓卉），她们分别毕业于北京大学新闻与传播学院和北京大学中文系。

这些节目不仅展现了高学历人才的多才多艺，同时也为观众带来了高质量的内容。

[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
最近，《非正式会谈》这档节目中出现了多位北京大学毕业的嘉宾。例如，节目中提到的一位飞行嘉宾纪向宁，正在北京大学攻读艺术系研究生，并且以439分的成绩位列北京大学艺术系研究生第一名。此外，该节目因其广泛的议题讨论和嘉宾的多样性而受到关注，其中不乏高学历背景的参与者，包括来自北京大学等知名学府的人士。

另外，脱口秀类节目《脱口秀大会》也是一个值得关注的例子，该节目中有多位演员拥有北京大学的教育背景，如李雪琴和鸟鸟（赵晓卉），她们分别毕业于北京大学新闻与传播学院和北京大学中文系。

这些节目不仅展现了高学历人才的多才多艺，同时也为观众带来了高质量的内容。
[AGENT] >>> Node 1: planner
**最终答案**
根据搜索结果，最近《非正式会谈》和《脱口秀大会》这两档节目中出现了较多北京大学毕业的演员。具体来说，《非正式会谈》中的纪向宁和《脱口秀大会》中的李雪琴及赵晓卉（鸟鸟）均毕业于北京大学。因此，这两个节目都可以视为近期有较多北大学员参与的节目。
[AGENT] >>> Node 2: observer


'根据搜索结果，最近《非正式会谈》和《脱口秀大会》这两档节目中出现了较多北京大学毕业的演员。具体来说，《非正式会谈》中的纪向宁和《脱口秀大会》中的李雪琴及赵晓卉（鸟鸟）均毕业于北京大学。因此，这两个节目都可以视为近期有较多北大学员参与的节目。'

## 3 多智能体协作 

illufly 也内置了多智能体支持方案。

### 3.1 顺序执行的多个智能体

In [21]:
from illufly.chat import ChatQwen
from illufly.flow import FlowAgent, End

flow = FlowAgent(
    ChatQwen(name="写手"),
    ChatQwen(name="翻译", memory=("system", "请你将我的作品翻译为英文")),
    End()
)

flow("帮我写一首关于兔子的四句儿歌?")

[AGENT] >>> Node 1: 写手
当然可以，这里有一首简单可爱的四句儿歌，适合小朋友唱：

小兔白又白，  
两耳竖起来。  
爱吃红萝卜，  
蹦蹦又跳跳。
[AGENT] >>> Node 2: 翻译
Sure, here is the translated version of the simple and cute four-line nursery rhyme, perfect for little ones to sing:

Little bunny white as snow,  
Ears stand up nice and tall, row.  
Loves to munch on carrots red,  
Hopping, skipping, full of fun instead.


'Sure, here is the translated version of the simple and cute four-line nursery rhyme, perfect for little ones to sing:\n\nLittle bunny white as snow,  \nEars stand up nice and tall, row.  \nLoves to munch on carrots red,  \nHopping, skipping, full of fun instead.'

### 3.2 两个智能体协作：一个创作一个打分

下面演示的两个智能体包含条件循环，如果「写手」写不出5分的作品，「打分专家」在打分后会要求写手继续写。

In [22]:
from illufly.chat import ChatQwen
from illufly.flow import FlowAgent, Selector

scorer = ChatQwen(
    name="打分专家",
    memory=[("system", "请你给我的作品打一个分数，从1分至5分，并给出改进意见。打分格式为:\n结果为x分")]
)

def should_continue():
    return "__END__" if "结果为5分" in scorer.last_output else "写手"

flow = FlowAgent(
    ChatQwen(name="写手"),
    scorer,
    Selector(condition=should_continue)
)

flow("你能帮我写一首关于兔子的四句儿歌?")

[AGENT] >>> Node 1: 写手
当然可以，这是一首简单的兔子儿歌：

小兔白又白，  
两耳竖起来。  
爱吃红萝卜，  
蹦跳真可爱。
[AGENT] >>> Node 2: 打分专家
结果为4分

这首儿歌非常简洁明快，符合儿童的审美和理解能力，易于记忆。语言上押韵，读起来朗朗上口。不过，如果能够加入一些互动性或教育意义的内容，比如增加一些小兔子的生活习性介绍，或者鼓励小朋友爱护小动物等，可能会让儿歌更加丰富和有意义。例如，可以添加一句“小朋友要爱它，轻轻抚摸不吵闹。”这样不仅增加了儿歌的内容，还传递了爱护动物的理念。
[AGENT] >>> Node 1: 写手
谢谢你的建议！我明白了，可以在儿歌中加入一些教育意义的内容。下面是改进后的版本：

小兔白又白，  
两耳竖起来。  
爱吃红萝卜，  
轻轻抚摸不吵闹。

希望这个版本更符合你的要求！
[AGENT] >>> Node 2: 打分专家
结果为5分

改进后的版本非常好！不仅保留了原来的简洁明快，还增加了教育意义，鼓励孩子们爱护小动物。这样的儿歌不仅能让孩子们更容易记住，还能培养他们的同理心和责任感。整体来说，这个版本更加丰富和有意义。

继续保持这种创作思路，相信你会写出更多优秀的作品！


'结果为5分\n\n改进后的版本非常好！不仅保留了原来的简洁明快，还增加了教育意义，鼓励孩子们爱护小动物。这样的儿歌不仅能让孩子们更容易记住，还能培养他们的同理心和责任感。整体来说，这个版本更加丰富和有意义。\n\n继续保持这种创作思路，相信你会写出更多优秀的作品！'

In [24]:
flow.agents[0][1].memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以，这是一首简单的兔子儿歌：\n\n小兔白又白，  \n两耳竖起来。  \n爱吃红萝卜，  \n蹦跳真可爱。'},
 {'role': 'user',
  'content': '结果为4分\n\n这首儿歌非常简洁明快，符合儿童的审美和理解能力，易于记忆。语言上押韵，读起来朗朗上口。不过，如果能够加入一些互动性或教育意义的内容，比如增加一些小兔子的生活习性介绍，或者鼓励小朋友爱护小动物等，可能会让儿歌更加丰富和有意义。例如，可以添加一句“小朋友要爱它，轻轻抚摸不吵闹。”这样不仅增加了儿歌的内容，还传递了爱护动物的理念。'},
 {'role': 'assistant',
  'content': '谢谢你的建议！我明白了，可以在儿歌中加入一些教育意义的内容。下面是改进后的版本：\n\n小兔白又白，  \n两耳竖起来。  \n爱吃红萝卜，  \n轻轻抚摸不吵闹。\n\n希望这个版本更符合你的要求！'}]